In [69]:
from time import sleep
from tqdm import tqdm

import numpy as np
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials

from keys import GOOG_API_KEY

In [70]:
## Load dataframes
# Polling averages
pres = pd.read_csv('averages/presidential_gen_approval.csv')
pres_issue = pd.read_csv('averages/presidential_issue_approvals.csv')
pres_rv = pd.read_csv('averages/presidential_RV_approval.csv')

# Polls display tables
pres_polls = pd.read_csv('transformed_tables/gen_approval_polls.csv')
issue_polls = pd.read_csv('transformed_tables/issue_approval_polls.csv')

In [71]:
# Define the scope
scopes = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]

# Authenticate with the service account
credentials = Credentials.from_service_account_file(
    GOOG_API_KEY,
    scopes=scopes
)

# Authorize gspread
client = gspread.authorize(credentials)

## Polling Averages

In [72]:
spreadsheet = client.open("snoutcounter-data")
worksheet = spreadsheet.worksheet("approval-stats")
worksheet.clear()

set_with_dataframe(worksheet, pres.reset_index())

In [73]:
worksheet = spreadsheet.worksheet("approval_stats_RV")
worksheet.clear()

set_with_dataframe(worksheet, pres_rv.reset_index())

In [74]:
issues = ['economy', 'immigration', 'inflation', 'trade_tariffs', 'foreign_policy', 'healthcare', 'crime']
issue_suffixes = ['econ', 'imm', 'infl', 'trade', 'fp', 'healthcare', 'crime']

for iss, suf in tqdm(zip(issues, issue_suffixes)):
    worksheet = spreadsheet.worksheet(f"approval_stats_{suf}")
    worksheet.clear()

    issue_df = pres_issue[pres_issue['issue'] == iss]
    set_with_dataframe(worksheet, issue_df.reset_index())

7it [00:05,  1.21it/s]


In [75]:
## Net issue approval
net_issues = pd.pivot_table(pres_issue[pres_issue['issue'].isin(issues)], 
                            values=['net', 'net_lower_ci', 'net_upper_ci'], index=['end_date'], columns=['issue'],
                           aggfunc="first").reset_index()
net_issues.columns = ['_'.join(col).strip() for col in net_issues.columns.values]
net_issues.head()

,end_date_,net_crime,net_economy,net_foreign_policy,net_healthcare,net_immigration,net_inflation,net_trade_tariffs,net_lower_ci_crime,net_lower_ci_economy,...,net_lower_ci_immigration,net_lower_ci_inflation,net_lower_ci_trade_tariffs,net_upper_ci_crime,net_upper_ci_economy,net_upper_ci_foreign_policy,net_upper_ci_healthcare,net_upper_ci_immigration,net_upper_ci_inflation,net_upper_ci_trade_tariffs
0,2025-01-21,0.0,7.918073,3.918073,0.000000,9.918073,-4.081927,-0.081927,0.0,7.918073,...,9.918073,-4.081927,-0.081927,0.0,7.918073,3.918073,0.000000,9.918073,-4.081927,-0.081927
1,2025-01-22,0.0,7.918073,3.918073,0.000000,9.918073,-4.081927,-0.081927,0.0,7.918073,...,9.918073,-4.081927,-0.081927,0.0,7.918073,3.918073,0.000000,9.918073,-4.081927,-0.081927
2,2025-01-23,0.0,2.633739,3.918073,-13.485259,3.206184,-4.081927,-0.081927,0.0,-3.924942,...,-5.124316,-4.081927,-0.081927,0.0,9.192420,3.918073,-13.485259,11.536684,-4.081927,-0.081927
3,2025-01-24,0.0,2.633739,3.918073,-13.485259,3.206184,-4.081927,-0.081927,0.0,-3.924942,...,-5.124316,-4.081927,-0.081927,0.0,9.192420,3.918073,-13.485259,11.536684,-4.081927,-0.081927
4,2025-01-25,0.0,2.633739,3.918073,-13.485259,3.206184,-4.081927,-0.081927,0.0,-3.924942,...,-5.124316,-4.081927,-0.081927,0.0,9.192420,3.918073,-13.485259,11.536684,-4.081927,-0.081927


In [76]:
worksheet = spreadsheet.worksheet("net_approval_issues")
worksheet.clear()

set_with_dataframe(worksheet, net_issues.reset_index())

## Polls Display Tables

In [77]:
pres_polls['sponsors'] = '^Sponsor: ' + pres_polls['sponsors'] + '^'
pres_polls['sponsors'] = pres_polls['sponsors'].fillna('')
pres_polls = pres_polls.sort_values(['total_weight'], ascending=False)
pres_polls = pres_polls.rename({
    'pollster':'Pollster', 'approve':'Approve', 'disapprove':'Disapprove', 'net':'Net', 
    'total_weight':'Weight', 'start_date':'Start Date', 'end_date':'End Date', 'url':'URL', 'adj_net':'Adjusted Net'}, axis='columns')
pres_polls['Sample'] = pres_polls['sample_size'].astype(int).astype(str) + ' ' + pres_polls['population']
pres_polls['Start Date'] = pd.to_datetime(pres_polls['Start Date']).astype(str)
pres_polls['End Date'] = pres_polls['End Date'].astype(str)
pres_polls = pres_polls.reset_index()
pres_polls['Pollster'] = ('<a href="' + pres_polls['URL'] + '" style="color:black; text-decoration:underline;"><u>' + 
                          pres_polls['Pollster'] + '</u></a>' + pres_polls['sponsors'])
pres_polls = pres_polls[['Pollster', 'Start Date', 'End Date', 'Sample', 'Weight', 'Approve', 'Disapprove', 'Net', 'Adjusted Net']]
pres_polls

,Pollster,Start Date,End Date,Sample,Weight,Approve,Disapprove,Net,Adjusted Net
0,"<a href=""https://pro.morningconsult.com/tracke...",2025-12-29,2026-01-04,45000 A,1.973645e-01,43.0,51.0,-8.0,-12.298873
1,"<a href=""https://www.reuters.com/world/america...",2026-01-04,2026-01-05,1248 A,1.722719e-01,42.0,56.0,-14.0,-10.044444
2,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-12-26,2025-12-29,1550 A,6.262818e-02,39.0,56.0,-17.0,-15.542064
3,"<a href=""https://www.atlasintel.org/poll/usa-n...",2025-12-15,2025-12-19,2315 A,5.886398e-02,39.3,59.6,-20.3,-20.831946
4,"<a href=""https://www.cbsnews.com/news/opinion-...",2025-12-17,2025-12-19,2300 A,4.414058e-02,41.0,59.0,-18.0,-16.542064
...,...,...,...,...,...,...,...,...,...
512,"<a href=""https://www.reuters.com/world/us/amer...",2025-01-24,2025-01-26,1034 A,1.211200e-18,45.0,46.0,-1.0,2.955556
513,"<a href=""https://pro.morningconsult.com/tracke...",2025-01-24,2025-01-26,2302 RV,1.154262e-18,52.0,44.0,8.0,3.667392
514,"<a href=""https://substack.com/inbox/post/15539...",2025-01-21,2025-01-21,742 A,9.367806e-19,49.0,36.0,13.0,8.192106
515,"<a href=""https://www.ipsos.com/en-us/reuters-i...",2025-01-20,2025-01-21,1077 A,7.144442e-19,47.0,41.0,6.0,9.955556


In [78]:
worksheet = spreadsheet.worksheet("polls_display_table")
worksheet.clear()

set_with_dataframe(worksheet, pres_polls)

In [79]:
issue_polls['sponsor'] = '^Sponsor: ' + issue_polls['sponsor'] + '^'
issue_polls['sponsor'] = issue_polls['sponsor'].fillna('')
issue_polls = issue_polls.sort_values(['issue', 'total_weight'], ascending=False)
issue_polls = issue_polls.rename({
    'pollster':'Pollster', 'approve':'Approve', 'disapprove':'Disapprove', 'net':'Net', 
    'total_weight':'Weight', 'start_date':'Start Date', 'end_date':'End Date', 'url':'URL', 'adj_net':'Adjusted Net',
    'issue':'Issue'}, axis='columns')
issue_polls['Sample'] = issue_polls['sample_size'].astype(int).astype(str) + ' ' + issue_polls['population']
issue_polls['Start Date'] = pd.to_datetime(issue_polls['Start Date']).astype(str)
issue_polls['End Date'] = issue_polls['End Date'].astype(str)
issue_polls = issue_polls.reset_index()
issue_polls['Pollster'] = ('<a href="' + issue_polls['URL'] + '" style="color:black; text-decoration:underline;"><u>' + 
                          issue_polls['Pollster'] + '</u></a>' + issue_polls['sponsor'])
issue_polls = issue_polls[issue_polls['Issue'].isin(issues)]
issue_polls['Issue'] = issue_polls['Issue'].replace({
    'economy': 'Economy',
    'immigration': 'Immigration',
    'foreign_policy': 'Foreign policy',
    'inflation': 'Inflation/prices',
    'trade_tariffs': 'Trade/tariffs',
    'healthcare': 'Healthcare',
    'crime': 'Crime'
})
issue_polls = issue_polls[['Pollster', 'Start Date', 'End Date', 'Issue', 'Sample', 'Weight', 'Approve', 'Disapprove', 'Net', 'Adjusted Net']]
issue_polls

,Pollster,Start Date,End Date,Issue,Sample,Weight,Approve,Disapprove,Net,Adjusted Net
48,"<a href=""https://www.atlasintel.org/poll/usa-n...",2025-12-15,2025-12-19,Trade/tariffs,2315 A,2.276824e-02,35.0,61.0,-26.0,-27.485259
49,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-12-20,2025-12-22,Trade/tariffs,1592 A,1.402810e-02,36.0,56.0,-20.0,-21.464147
50,"<a href=""https://pro.morningconsult.com/tracke...",2025-12-19,2025-12-21,Trade/tariffs,2203 RV,1.362925e-02,45.0,46.0,-1.0,-13.305970
51,"<a href=""https://poll.qu.edu/poll-release?rele...",2025-12-11,2025-12-15,Trade/tariffs,1035 RV,1.094028e-02,40.0,55.0,-15.0,-13.157660
52,"<a href=""https://static.foxnews.com/foxnews.co...",2025-12-12,2025-12-15,Trade/tariffs,1001 RV,1.023853e-02,36.0,64.0,-28.0,-26.562556
...,...,...,...,...,...,...,...,...,...,...
1170,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-02-23,2025-02-25,Crime,1604 A,3.377123e-13,47.0,37.0,10.0,8.535853
1171,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-02-16,2025-02-18,Crime,1603 A,1.947101e-13,48.0,37.0,11.0,9.535853
1172,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-02-09,2025-02-11,Crime,1595 A,1.125270e-13,45.0,41.0,4.0,2.535853
1173,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-02-02,2025-02-04,Crime,1604 A,6.593942e-14,48.0,35.0,13.0,11.535853


In [80]:
worksheet = spreadsheet.worksheet("polls_display_issues")
worksheet.clear()

set_with_dataframe(worksheet, issue_polls)